In [1]:
from sage.matrix.berlekamp_massey import berlekamp_massey
from sage.matrix.special import *
import random
def rand_list(n, card):
    return [random.randint(0, card) for _ in range(n)]

In [2]:
# n = 100, λ (e) = random.randint(q)
ref = ['D100', 'DRandom', 'J100', '10*J10', 'J50 + j50', 'J50 + J30 + J20']
def mat_gen(q, n=10):
    e1 = random.randint(1, q)
    e2 = random.randint(1, q)
    while e2 == e1: 
        e2 = random.randint(1, q)
    A = diagonal_matrix(GF(q), [e1]*n)
    B = diagonal_matrix(GF(q), [random.randint(0, q) for _ in range(n)])
    C = jordan_block(e1, n)
    D = block_diagonal_matrix([jordan_block(e1, n//10)]*10)
    E = block_diagonal_matrix([jordan_block(e1, n//2), jordan_block(e2, n//2)])
    F = block_diagonal_matrix([jordan_block(e1, n//2), jordan_block(e1, n//10*3), jordan_block(e1, n//5)])
    return [A, B, C, D, E, F]
ls = mat_gen(7)
for A in ls:
    print(A)
    print()

[3 0 0 0 0 0 0 0 0 0]
[0 3 0 0 0 0 0 0 0 0]
[0 0 3 0 0 0 0 0 0 0]
[0 0 0 3 0 0 0 0 0 0]
[0 0 0 0 3 0 0 0 0 0]
[0 0 0 0 0 3 0 0 0 0]
[0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 3 0 0]
[0 0 0 0 0 0 0 0 3 0]
[0 0 0 0 0 0 0 0 0 3]

[2 0 0 0 0 0 0 0 0 0]
[0 2 0 0 0 0 0 0 0 0]
[0 0 6 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 5 0 0 0 0 0]
[0 0 0 0 0 2 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 2 0]
[0 0 0 0 0 0 0 0 0 0]

[3 1 0 0 0 0 0 0 0 0]
[0 3 1 0 0 0 0 0 0 0]
[0 0 3 1 0 0 0 0 0 0]
[0 0 0 3 1 0 0 0 0 0]
[0 0 0 0 3 1 0 0 0 0]
[0 0 0 0 0 3 1 0 0 0]
[0 0 0 0 0 0 3 1 0 0]
[0 0 0 0 0 0 0 3 1 0]
[0 0 0 0 0 0 0 0 3 1]
[0 0 0 0 0 0 0 0 0 3]

[3|0|0|0|0|0|0|0|0|0]
[-+-+-+-+-+-+-+-+-+-]
[0|3|0|0|0|0|0|0|0|0]
[-+-+-+-+-+-+-+-+-+-]
[0|0|3|0|0|0|0|0|0|0]
[-+-+-+-+-+-+-+-+-+-]
[0|0|0|3|0|0|0|0|0|0]
[-+-+-+-+-+-+-+-+-+-]
[0|0|0|0|3|0|0|0|0|0]
[-+-+-+-+-+-+-+-+-+-]
[0|0|0|0|0|3|0|0|0|0]
[-+-+-+-+-+-+-+-+-+-]
[0|0|0|0|0|0|3|0|0|0]
[-+-+-+-+-+-+-+-+-+-]
[0|0|0|0|0|0|0|3|0|0]
[-+-+-+

In [3]:
def krylov(u, v, A, m, start = 0):
    return [u*(A^i*v) for i in range(start, m)]

def random_vector(n, f):
    return vector(f, [f.random_element() for _ in range(n)])

In [4]:
def early_BM(q, sequence, n, c=3):
    ω = c*log(n, q) + 1
    f = GF(q)
    R = PolynomialRing(f, "x")
    sequence = [f(s) for s in sequence]
    C = [f(1)]
    B = [f(1)]
    L = 0
    m = 1
    streak = 1
    b = f(1)
    stop = 0
    for k in range(len(sequence)):
        d = sequence[k] + sum(C[i] * sequence[k - i] for i in range(1, L + 1))
        if d == 0:
            m += 1
            streak += 1
            if streak >= ω:
                stop = k
                break
        elif 2 * L <= k:
            T = C[:]
            C += [f(0)] * (m + len(B) - len(C))
            for i in range(len(B)):
                C[i + m] -= d / b * B[i]
            L = k + 1 - L
            B = T
            b = d
            m = 1
            streak = 1
        else:
            C += [f(0)] * (m + len(B) - len(C))
            for i in range(len(B)):
                C[i + m] -= d / b * B[i]
            m += 1
            streak = 1
    else: 
        stop = 2*n
    # result processing before return
    C = C[::-1]
    while len(C) > 1 and C[0] == 0:
        C = C[1:]
    return R(C), stop

In [5]:
import matplotlib.pyplot as plt

def plot_success(success_map, x_list, x_lab = 'x'):
    min_success = 1
    num_plots = len(success_map)
    rows = 2
    cols = 3
    # fig, axes = plt.subplots(rows, cols, figsize=(15, 8))
    fig, axes = plt.subplots(rows, cols, figsize=(16, 8))
    axes = axes.flatten()
    for idx, (mat_type, success_list) in enumerate(success_map.items()):
        min_success = min(min_success, min(success_list))
        ax = axes[idx]
        ax.plot(x_list, success_list)
        ax.set_title(mat_type)
        ax.set_ylim(0.5, 1.1)
        ax.set_xlabel(x_lab)
        ax.set_ylabel("Prob.")
    for j in range(idx + 1, len(axes)):
        fig.delaxes(axes[j])
    fig.suptitle('Success Probability by Matrix Type', fontsize=16, y=1.02)
    plt.tight_layout()
    plt.show()
    print(f'Min success rate: {float(min_success):.3f}')

def plot_stops(stops_map, x_list, comp_list, x_lab = 'x', y_lim = None):
    num_plots = len(stops_map)
    rows = 2
    cols = 3
    fig, axes = plt.subplots(rows, cols, figsize=(16, 8))
    axes = axes.flatten()
    for idx, (mat_type, stops_list) in enumerate(stops_map.items()):
        ax = axes[idx]
        ax.plot(x_list, stops_list, label='Early term. at iter. k')
        ax.plot(x_list, comp_list, 'r--', label='Standard term. at iter. 2n')
        ax.set_title(mat_type)
        ax.set_xlabel(x_lab)
        ax.set_ylabel("k")
        if y_lim:
            ax.set_ylim(y_lim)
        ax.legend()
    for j in range(idx + 1, len(axes)):
        fig.delaxes(axes[j])
    fig.suptitle('Avg. Stop by Matrix Type', fontsize=16, y=1.02)
    plt.tight_layout()
    plt.show()

In [6]:
def varyingN(q = 5, n_lim = 51, trials = 100):
    n_list = [10*i for i in range(1, n_lim)]
    f = GF(q)
    avg_stops = {}
    success_rates = {}
    for n in n_list:
        print(n)
        matlist = mat_gen(q, n)
        for i in range(6):
            sum_stop = 0
            failed = 0
            A = matlist[i]
            mat_type = ref[i]
            for _ in range(trials):
                u = random_vector(n, f)
                v = random_vector(n, f)
                k = krylov(u, v, A, 2*n)
                b_temp, stop = early_BM(q, k, n)
                sum_stop += stop
                b = berlekamp_massey(k)
                if b!=b_temp:
                    failed += 1
            success_rates.setdefault(mat_type, []).append(1 - failed / trials)
            avg_stops.setdefault(mat_type, []).append(sum_stop / trials)
    plot_success(success_rates, n_list, x_lab = 'n')
    comp_list = [2*i for i in n_list]
    plot_stops(avg_stops, n_list, comp_list, x_lab = 'n')
    # return avg_stops, success_rates

In [7]:
def varyingQ(n = 100, q_lim = 100, trials = 100):
    q_list = prime_range(0, q_lim)
    avg_stops = {}
    success_rates = {}
    for q in q_list:
        print(q)
        f = GF(q)
        matlist = mat_gen(q, n)
        for i in range(len(matlist)):
            sum_stop = 0
            failed = 0
            A = matlist[i]
            mat_type = ref[i]
            for _ in range(trials):
                u = random_vector(n, f)
                v = random_vector(n, f)
                k = krylov(u, v, A, 2*n)
                b_temp, stop = early_BM(q, k, n)
                sum_stop += stop
                b = berlekamp_massey(k)
                if b!=b_temp:
                    failed += 1
            success_rates.setdefault(mat_type, []).append(float(1 - failed / trials))
            avg_stops.setdefault(mat_type, []).append(float(sum_stop / trials))
    plot_success(success_rates, q_list, x_lab = 'q')
    comp_list = [2*n]*len(q_list)
    plot_stops(avg_stops, q_list, comp_list, x_lab = 'q', y_lim = (0, 2*n + 1))
    # return avg_stops, success_rates

In [ ]:
varyingN()

10
20
30
40
50


In [ ]:
varyingQ()

In [ ]:
# check jordan's success rate
# varying jordan block size & diagonals
# wiedemann's min poly success with same test cases